In [42]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [43]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [44]:
df.shape

(569, 33)

In [45]:
df.drop(columns=["id", "Unnamed: 32"], inplace=True)

In [46]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [47]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0], test_size=.2, random_state=42)

In [48]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [49]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [50]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [51]:
X_test_tensor.shape

torch.Size([114, 30])

In [52]:
class NN():
    def __init__(self, X):
        self.weights = torch.rand(X.shape[1], 1, dtype = torch.float64, requires_grad=True)
        self.bias = torch.zeros(1, dtype= torch.float64, requires_grad=True)
    
    def forward(self, X):
        z = torch.matmul(X, self.weights) + self.bias
        y_pred = torch.sigmoid(z)
        return y_pred

    def loss(self,y_pred, y):
        epsilion = 1e-7
        y_pred = torch.clamp(y_pred, epsilion, 1-epsilion)

        loss = -(y_train_tensor * torch.log(y_pred) + (1-y_train_tensor) * torch.log(1-y_pred)).mean()
        return loss

In [53]:
lr = 0.1
epochs = 250


In [54]:
model = NN(X_train_tensor)

for epoch in range(epochs):
    y_pred = model.forward(X_train_tensor)
    
    loss = model.loss(y_pred, y_train_tensor)

    loss.backward()

    with torch.no_grad():
        model.weights -= lr * model.weights.grad
        model.bias -= lr * model.bias.grad

    model.weights.grad.zero_()
    model.bias.grad.zero_()
    
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 3.445819318191677
Epoch 2, Loss: 3.312649590506003
Epoch 3, Loss: 3.1722233473108528
Epoch 4, Loss: 3.0268440502997715
Epoch 5, Loss: 2.8678134811132945
Epoch 6, Loss: 2.7053984776939606
Epoch 7, Loss: 2.543507114561458
Epoch 8, Loss: 2.3792995783316786
Epoch 9, Loss: 2.214323684929525
Epoch 10, Loss: 2.048323993503431
Epoch 11, Loss: 1.881001193945399
Epoch 12, Loss: 1.7212821931695494
Epoch 13, Loss: 1.5724345946774312
Epoch 14, Loss: 1.4360225125174422
Epoch 15, Loss: 1.311929737870045
Epoch 16, Loss: 1.2012441304806352
Epoch 17, Loss: 1.106409923482133
Epoch 18, Loss: 1.027392132224836
Epoch 19, Loss: 0.9633284790996507
Epoch 20, Loss: 0.9125267238215845
Epoch 21, Loss: 0.8726998400509761
Epoch 22, Loss: 0.8414681786745974
Epoch 23, Loss: 0.8167841813973008
Epoch 24, Loss: 0.797058032420011
Epoch 25, Loss: 0.7811074243640438
Epoch 26, Loss: 0.7680658361460689
Epoch 27, Loss: 0.7572970330466929
Epoch 28, Loss: 0.7483275806198434
Epoch 29, Loss: 0.7407982795694946
Epoc

In [57]:
with torch.no_grad():
    y_pred = model.forward(X_test_tensor)
    y_pred = (y_pred > 0.5).float()
    accuracy = (y_pred == y_test_tensor).float().mean()
    

In [58]:
accuracy

tensor(0.6228)